In [ ]:
import os
import os
import numpy as np
import pickle
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import re
import time
from Util import Env, quantizer, shifter, get_parser
datadir = 'lotr/04-crypto-sc/data/'

In [ ]:
onefiles = os.listdir(datadir + 'eddsa-1-parsed')
onelist = []
for fname in onefiles:
    with open(datadir + 'eddsa-1-parsed/' + fname, 'r') as f:
        onelist.append([int(n) for n in f])
print(len(onelist))
onethreshold = np.percentile(np.array([len(l) for l in onelist]),0.3)

zerofiles = os.listdir(datadir + 'eddsa-0-parsed')
zerolist = []
for fname in zerofiles:
    with open(datadir + 'eddsa-0-parsed/' + fname, 'r') as f:
        zerolist.append([int(n) for n in f])
print(len(zerolist))
zerothreshold = np.percentile(np.array([len(l) for l in zerolist]),0.3)
print((onethreshold, zerothreshold))

In [ ]:
threshold = int(min(zerothreshold, onethreshold))
newzero = [z for z in zerolist if len(z) >= threshold]
newone = [o for o in onelist if len(o) >= threshold]
dsize = min((len(newzero),len(newone)))
x_arr = np.zeros([dsize*2, threshold],dtype=np.float32)
y_arr = np.arange(dsize*2)%2
for i in range(dsize):
    x_arr[2*i] = newzero[i][:threshold]
    x_arr[2*i+1] = newone[i][:threshold]
print(x_arr.shape)

In [ ]:
std = 16
mean = np.median(x_arr)
x_arr2 = (x_arr-mean)/std
print((mean,))

In [ ]:
train_x = x_arr2[:2000]
train_y = y_arr[:2000]
valid_x = x_arr2[2000:4000]
valid_y = y_arr[2000:4000]
test_x = x_arr2[2000:5000]
test_y = y_arr[2000:5000]
from sklearn import svm
clf = svm.SVC(kernel = 'rbf',gamma=0.1, C=0.5)

clf.fit(train_x, train_y)
pred_y = clf.predict(test_x)
#tpred_y = clf.predict(train_x)
(pred_y == test_y).sum()/len(pred_y)

In [ ]:
if os.path.isfile("eddsa.pkl"):
    x_arr_old, y_arr_old = pickle.load(open("eddsa.pkl",'rb'))
    newlen = min(x_arr_old.shape[1], x_arr.shape[1])
    print("Old tracelen {}, new tracelen {}".format(x_arr_old.shape[1], newlen))
    x_arr_new = np.concatenate([x_arr_old[:,:newlen], x_arr[:,:newlen]], axis=0)
    y_arr_new = np.concatenate([y_arr_old, y_arr])
else:
    x_arr_new = x_arr
    y_arr_new = y_arr
print("total num traces: " + str(len(y_arr_new)))
traincut = 7*(len(y_arr_new)//10)
valcut = 17*(len(y_arr_new)//20)
pickle.dump((x_arr_new, y_arr_new), open("eddsa.pkl", "wb"))
pickle.dump((x_arr_new[:traincut], y_arr_new[:traincut]), open("eddsa_train.pkl", "wb"))
pickle.dump((x_arr_new[traincut:valcut], y_arr_new[traincut:valcut]), open("eddsa_valid.pkl", "wb"))
pickle.dump((x_arr_new[valcut:], y_arr_new[valcut:]), open("eddsa_test.pkl", "wb"))

In [ ]:
onefiles = os.listdir(datadir + 'rsa-1-parsed')
onelist = []
for fname in onefiles:
    with open(datadir + 'rsa-1-parsed/' + fname, 'r') as f:
        onelist.append([int(n) for n in f])
print(len(onelist))
onethreshold = np.percentile(np.array([len(l) for l in onelist]),0.3)

zerofiles = os.listdir(datadir + 'rsa-0-parsed')
zerolist = []
for fname in zerofiles:
    with open(datadir + 'rsa-0-parsed/' + fname, 'r') as f:
        zerolist.append([int(n) for n in f])
print(len(zerolist))
zerothreshold = np.percentile(np.array([len(l) for l in zerolist]),0.3)
print((onethreshold, zerothreshold))

In [ ]:
threshold = int(min(zerothreshold, onethreshold))
newzero = [z for z in zerolist if len(z) >= threshold]
newone = [o for o in onelist if len(o) >= threshold]
dsize = min((len(newzero),len(newone)))
x_arr = np.zeros([dsize*2, threshold],dtype=np.float32)
y_arr = np.arange(dsize*2)%2
for i in range(dsize):
    x_arr[2*i] = newzero[i][:threshold]
    x_arr[2*i+1] = newone[i][:threshold]
print(x_arr.shape)

std = 16
mean = np.median(x_arr)
x_arr2 = (x_arr-mean)/std
print((mean,))

In [ ]:
train_x = x_arr2[:7000]
train_y = y_arr[:7000]
valid_x = x_arr2[7000:8500]
valid_y = y_arr[7000:8500]
test_x = x_arr2[8500:10000]
test_y = y_arr[8500:10000]
from sklearn import svm
clf = svm.SVC(kernel = 'rbf',gamma=0.01, C=10.0)

clf.fit(train_x, train_y)
pred_y = clf.predict(test_x)
#tpred_y = clf.predict(train_x)
(pred_y == test_y).sum()/len(pred_y)

In [ ]:
pickle.dump((x_arr, y_arr), open("rsa.pkl", "wb"))
pickle.dump((x_arr[:7000], y_arr[:7000]), open("rsa_train.pkl", "wb"))
pickle.dump((x_arr[7000:8500], y_arr[7000:8500]), open("rsa_valid.pkl", "wb"))
pickle.dump((x_arr[8500:10000], y_arr[8500:10000]), open("rsa_test.pkl", "wb"))